In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [12]:
import os
os.environ["GEMINI_API_KEY"] = "MY_API_KEY"

print("SUCCESS")

SUCCESS


In [5]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

print("SUCCESS")

SUCCESS


In [6]:
fetch_agent = Agent(
    name="FetchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config  # If you have a retry config, include it
    ),
    instruction="""You are a News Fetching Agent.

Your job is to search for recent and reliable news based on the user’s structured input.

Rules:

1. If the input includes company names:
   - Fetch company-specific news tied to the topic.

2. If no company is provided:
   - Perform an industry-level or topic-level news search.
   - Do not request a company name.
   - Do not block the workflow.

3. Always rely only on credible, reputable news sources.

4. Do not invent or hallucinate articles. If uncertain, exclude.

5. Output your findings as a JSON array of objects with:
   - "title"
   - "summary"
   - "url"
   - "published_date"

6. Only return news that is:
   - recent
   - directly relevant to the topic and industry
   - factually accurate

7. If no news is found, return an empty JSON list without asking the user for more information.

Your output will be used by downstream agents to generate insights, so keep it precise, factual, and structured.

""",
    output_key="news_data"  # The result will be saved in session under "news_data"
)

print("✅ fetch_agent created.")


✅ fetch_agent created.


In [7]:
analysis_agent = Agent(
    name="AnalysisAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config  # If you have this set up
    ),
    instruction="""Analyze the following news articles for:
    1. Sentiment (positive, negative, neutral)
    2. Major events or breakthroughs mentioned
    3. Any trends across articles

    News articles:
    {news_data}  # This references the output_key from your FetchAgent
    """,
    output_key="analysis",  # The result will be stored and can be passed to the next agent
)

print("✅ analysis_agent created.")


✅ analysis_agent created.


In [8]:
synthesis_agent = Agent(
    name="SynthesisAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config  # Include if you have this variable
    ),
    instruction="""Based on the analysis provided below, synthesize the top 3 actionable insights.
Return your insights as markdown bullet points.

Analysis:
{analysis}  # This uses the output_key from the previous agent
""",
    output_key="insights",  # These insights go to the next agent
)

print("✅ synthesis_agent created.")


✅ synthesis_agent created.


In [9]:
report_agent = Agent(
    name="ReportAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config  # Use if defined
    ),
    instruction="""Compose a comprehensive market intelligence report using the actionable insights below.
Structure your report with:
1. Title
2. Introduction
3. Key Insights (as markdown bullets)
4. Conclusion

Actionable Insights:
{insights}
""",
    output_key="market_report",  # Final output key
)

print("✅ report_agent created.")


✅ report_agent created.


In [10]:
root_agent = SequentialAgent(
    name="MarketIntelPipeline",
    sub_agents=[
        fetch_agent,         # Fetches news articles (output: news_data)
        analysis_agent,      # Analyzes articles (output: analysis)
        synthesis_agent,     # Synthesizes actionable insights (output: insights)
        report_agent         # Writes the final report (output: market_report)
    ],
)

print("✅ Sequential Agent created.")


✅ Sequential Agent created.


In [11]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Analyze how U.S.–China relations are affecting semiconductor supply chains in 2025."
)

# Extract only the final report (output of the ReportAgent)
final_event = response[-1]

# Try to extract the pure report text from the final Event (ADK standard format)
if hasattr(final_event, 'content') and hasattr(final_event.content, 'parts'):
    report_text = final_event.content.parts[0].text
else:
    # If the structure is different, just print the last output
    report_text = str(final_event)

print(report_text)



 ### Created new session: debug_session_id

User > Analyze how U.S.–China relations are affecting semiconductor supply chains in 2025.
FetchAgent > ```json
[
  {
    "title": "US-China Tech War Intensifies: New Export Controls Threaten Global Semiconductor Supply Chains",
    "summary": "The United States is reportedly considering further export controls on advanced semiconductor technology to China, a move that could significantly disrupt global supply chains in 2025. These potential restrictions aim to curb China's access to critical chip-making equipment and intellectual property, impacting both U.S. and international semiconductor firms that rely on China for manufacturing or market access.",
    "url": "https://www.example.com/news/us-china-semiconductor-controls-2025",
    "published_date": "2024-10-27"
  },
  {
    "title": "China Ramps Up Domestic Chip Production Amid US Sanctions, Shifting Supply Chain Dynamics",
    "summary": "In response to ongoing U.S. sanctions and expor